# Santa Movie Challenge as Travelling Salesman Problem

This notebook is an attempt to improve the best currently published solution that doesn't use wildcards by peeking closely to repeating permutations in it and trying to rebalance them.

# Load the solution

At the moment of writing this notebook such a solution is the [one](https://www.kaggle.com/cdeotte/santa-2021-tsp-baseline-2500/notebook) created by the [Chris Deotte](https://www.kaggle.com/cdeotte). Let's load it. 

In [ ]:
import itertools
import numpy as np
import pandas as pd


!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz
!tar xvfz LKH-3.0.7.tgz
!cd LKH-3.0.7; make; cp LKH ..

In [ ]:
LETTERS = {
    1: '🎅',  # father christmas
    2: '🤶',  # mother christmas
    3: '🦌',  # reindeer
    4: '🧝',  # elf
    5: '🎄',  # christmas tree
    6: '🎁',  # gift
    7: '🎀',  # ribbon
    8: '🌟',  # star
}
INV_LETTERS = {v: k for k, v in LETTERS.items()}

solution = pd.read_csv('../input/santa-2021-tsp-baseline-2500/submission.csv')
strings = [[INV_LETTERS[c] for c in s] for s in solution.schedule]
strings.sort(key=len, reverse=True)
print(f'Strings lengths are {[len(_) for _ in strings]}.')

# Peek closely at the permutations in each string

In [ ]:
def find_strings_perms(strings, verbose=False):
    all_perms = set(itertools.permutations(range(1, 8), 7))
    perms = []
    for s in strings:
        perms.append([])
        for i in range(len(s)-6):
            p = tuple(s[i:i+7])
            if p in all_perms:
                perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return perms

strings_perms = find_strings_perms(strings, verbose=True)

# Permutations rebalancing

5772 permutations and 5709 unique permutations is more than 5280 that should be placed in the strings, there are some duplicates, let's leave each duplicate in the shortest string it occurs in.

In [ ]:
def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms] 
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != (1, 2) and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != (1, 2) and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms] 

strings_perms = rebalance_perms(strings_perms, verbose=True)

Well, 1647 and 1696 is much less than 1884 and 1888. Let's feed it to the LKH solver again.

# Use LKH to solve ATSP

We'll use [LKH-3](http://webhotel4.ruc.dk/~keld/research/LKH-3/) to try to improve current solution and will pass current solution to LKH as initial tour to start optimization from.

We'll keep rebalancing permutations and feeding it again to LKH while improvements are made.

In [ ]:
def perm_dist(p, q):
    i = p.index(q[0])
    return i if p[i:] == q[:7-i] else 7

def perms_to_string(perms):
    perms = list(perms)
    s = [*perms[0]]
    for p, q in zip(perms, perms[1:]):
        d = perm_dist(p, q)
        s.extend(q[-d:])
    return s

def distances_matrix(perms):
    m = np.zeros((len(perms), len(perms)), dtype='int8')
    for i, p in enumerate(perms):
        for j, q in enumerate(perms):
            m[i, j] = perm_dist(p, q)
    return m

def write_params_file():
    with open('santa.par', 'w') as f:
        print('PROBLEM_FILE = santa.atsp', file=f)
        print('TOUR_FILE = best_tour.txt', file=f)
        print('INITIAL_TOUR_FILE = initial_tour.txt', file=f)
        print('PATCHING_C = 4', file=f)
        print('PATCHING_A = 3', file=f)
        print('GAIN23 = YES', file=f)
        print('SEED = 42', file=f)
        print('MAX_TRIALS = 100000', file=f)
        print('TIME_LIMIT = 600', file=f) #seconds
        print('TRACE_LEVEL = 1', file=f)

def write_problem_file(distances):
    with open('santa.atsp', 'w') as f:
        print('TYPE: ATSP', file=f)
        print(f'DIMENSION: {len(distances)}', file=f)
        print('EDGE_WEIGHT_TYPE: EXPLICIT', file=f)
        print('EDGE_WEIGHT_FORMAT: FULL_MATRIX\n', file=f)
        print('EDGE_WEIGHT_SECTION', file=f)
        for row in distances:
            print(' '.join(str(_) for _ in row), file=f)

def write_initial_tour_file(perms):
    with open('initial_tour.txt', 'w') as f:
        print('TOUR_SECTION', file=f)
        print(' '.join(str(_) for _ in range(1, len(perms)+1)), -1, file=f)

def read_output_tour(perms):
    perms = list(perms)
    with open('best_tour.txt') as f:
        lines = f.readlines()
    tour = lines[lines.index('TOUR_SECTION\n')+1:-2]
    return [perms[int(_) - 1] for _ in tour] 
    
def solve_atsp(perms, verbose=False):
    write_params_file()
    distances = distances_matrix(perms)
    write_problem_file(distances)
    write_initial_tour_file(perms)
    
    # Run LKH-3 to solve ATSP instance
    if verbose:
        !./LKH santa.par
    else:
        !touch lkh.log
        !./LKH santa.par >> lkh.log
    tour = read_output_tour(perms)
    return perms_to_string(tour)

In [ ]:
improved, old_lens = True, [len(_) for _ in strings]
while improved:
    print('='*91)
    new_strings = [solve_atsp(_) for _ in strings_perms]
    new_strings.sort(key=len, reverse=True)
    new_lens = [len(_) for _ in new_strings]
    if new_lens < old_lens:
        print(f'Improved strings lengths from {old_lens} to {new_lens}.')
        strings, old_lens = new_strings, new_lens
        strings_perms = find_strings_perms(strings, verbose=True)
        strings_perms = rebalance_perms(strings_perms, verbose=True)
    else:
        improved = False

Wow, we improved the solution quite a bit. Let's check it's the correct one and we haven't lost any permutations along the way.

In [ ]:
all_perms = set(itertools.permutations(range(1, 8), 7))
mandatory_perms = set((1, 2) +  _ for _ in itertools.permutations(range(3, 8), 5))

strings_perms = [set(_) for _ in find_strings_perms(strings)]
for i, s in enumerate(strings_perms):
    if mandatory_perms - s:
        print(f'String #{i} is missing {mandatory_perms - s}.')
if all_perms - set.union(*strings_perms):
    print(f'Strings are missing {all_perms - set.union(*strings_perms)}.')

As everything is ok, now we'll save the found solution and then try to improve it even more using wildcards.

In [ ]:
sub = pd.DataFrame()
sub['schedule'] = [''.join(LETTERS[x] for x in s) for s in strings]
sub_name = f'submission_no_wildcards_{"_".join(str(len(_)) for _ in strings)}.csv'
sub.to_csv(sub_name, index=False)

# Wildcards optimization

We'll use the code from the [notebook](https://www.kaggle.com/yosshi999/wildcard-postprocessing-using-dynamic-programming) created by [Yosshi999](https://www.kaggle.com/yosshi999) to improve our solution with wildcards.

In [ ]:
import torch
import torch.nn.functional as F


perms = list(map(lambda p: "".join(p), itertools.permutations("1234567")))
perm2id = {p: i for i, p in enumerate(perms)}

perms_arr = np.array([list(map(int, p)) for p in perms])
perms_arr.shape

perms_onehot = np.eye(7)[perms_arr-1, :].transpose(0, 2, 1)
assert np.allclose(perms_onehot[:,0,:].astype(np.int64), (perms_arr == 1).astype(np.int64))

# print("onehot 1234567:")
# print(perms_onehot[perm2id["1234567"]])

# print("onehot 5671234:")
# print(perms_onehot[perm2id["5671234"]])

# print("correlate between 1234567 and 5671234")
left = perms_onehot[perm2id["1234567"]]
right = perms_onehot[perm2id["5671234"]]
matches = F.conv2d(
    F.pad(torch.Tensor(left[None, None, :, :]), (7, 7)),
    torch.Tensor(right[None, None, :, :]),
    padding="valid"
).numpy().reshape(-1)
# print(matches)
must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_right2left = np.array([0, 1, 2, 3, 4, 5, 6, 7, -1, -1, -1, -1, -1, -1, -1])
cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])
# print("cost of 1234567 -> 5671234:", min(cost_ifmatch[np.equal(must_match_left2right, matches)]))
# print("cost of 5671234 -> 1234567:", min(cost_ifmatch[np.equal(must_match_right2left, matches)]))


M = F.conv2d(
    F.pad(torch.Tensor(perms_onehot[:, None, :, :]), (7, 7)),
    torch.Tensor(perms_onehot[:, None, :, :]),
    padding="valid"
).squeeze().numpy()

# M.shape


must_match_left2right = np.array([-1, -1, -1, -1, -1, -1, -1, 7, 6, 5, 4, 3, 2, 1, 0])
must_match_left2right_wild = np.array([-1, -1, -1, -1, -1, -1, -1, 6, 5, 4, 3, 2, 1, 0, 0])

cost_ifmatch = np.array([7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7])

costMat = np.where(M == must_match_left2right, cost_ifmatch, np.inf).min(axis=-1).astype(np.int8)
costMatWild = np.minimum(costMat, np.where(M == must_match_left2right_wild, cost_ifmatch, np.inf).min(axis=-1)).astype(np.int8)

symbols = "🎅🤶🦌🧝🎄🎁🎀"
schedule = sub.schedule
words = [s.translate(str.maketrans(symbols, "1234567")) for s in schedule]

nodes_list = []
table_list = []
for i in range(3):
    word = words[i]
    nodes = []
    for i in range(len(word)-6):
        p = word[i:i+7]
        if p in perm2id:
            nodes.append(perm2id[p])
    table = np.zeros((len(nodes), 10), np.int64)
    table[0, :] = 7
    for i in range(1, len(nodes)):
        e = costMat[nodes[i-1], nodes[i]]
        ew = costMatWild[nodes[i-1], nodes[i]]
        table[i,0] = table[i-1,0] + e
        table[i,1] = min(table[i-1,1] + e, table[i-1,0] + ew)
        table[i,2] = min(table[i-1,2], table[i-1,1]) + e # TODO: better transition
        table[i,3] = min(table[i-1,3], table[i-1,2]) + e
        table[i,4] = min(table[i-1,4], table[i-1,3]) + e
        table[i,5] = min(table[i-1,5], table[i-1,4]) + e
        table[i,6] = min(table[i-1,6], table[i-1,5]) + e
        table[i,7] = min(table[i-1,7], table[i-1,6]) + e
        table[i,8] = min(table[i-1,8], table[i-1,7]) + e
        table[i,9] = min(table[i-1,9] + e, table[i-1,8] + ew)
#     print(table[-1].min(), table[-1])
    nodes_list.append(nodes)
    table_list.append(table)
    
# backtrack
new_words = []
wilds = []
for nodes, table in zip(nodes_list, table_list):
    ns = [perms[nodes[-1]]]
    track = np.argmin(table[-1])
    wild = []
    for i in range(len(nodes)-2, -1, -1):
        e = costMat[nodes[i], nodes[i+1]]
        ew = costMatWild[nodes[i], nodes[i+1]]
        if track == 0:
            ns.append(perms[nodes[i]][:e])
        elif track == 1:
            if table[i, 1] + e < table[i, 0] + ew:
                ns.append(perms[nodes[i]][:e])
            else:
                left = np.array(list(map(int, perms[nodes[i]][ew:])))
                right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                mis = np.where(left != right)[0][0]
                wild.append(table[i, track-1]-7+ew+mis)
                ns.append(perms[nodes[i]][:ew])
                track = track - 1
        elif 2 <= track <= 8:
            if table[i, track] >= table[i, track-1]:
                track = track - 1
            ns.append(perms[nodes[i]][:e])
        elif track == 9:
            if table[i, 9] + e < table[i, 8] + ew:
                ns.append(perms[nodes[i]][:e])
            else:
                ns.append(perms[nodes[i]][:ew])
                left = np.array(list(map(int, perms[nodes[i]][ew:])))
                right = np.array(list(map(int, perms[nodes[i+1]][:-ew])))
                mis = np.where(left != right)[0][0]
                wild.append(table[i, track-1]-7+ew+mis)
                track = track - 1
        else:
            assert False
    assert track == 0
    wilds.append(wild)
    nsw = list("".join(ns[::-1]))
    for w in wild:
        nsw[w] = "8"
    new_words.append("".join(nsw))
    
strings = [[int(_) for _ in s] for s in new_words]
strings.sort(key=len, reverse=True)
new_lens = [len(_) for _ in strings]
print(f'Improved strings lengths from {old_lens} to {new_lens}.')

We've used wildcards, let's save this solution.

In [ ]:
sub = pd.DataFrame()
sub['schedule'] = [''.join(LETTERS[x] for x in s) for s in strings]
sub_name = f'submission_with_wildcards_{"_".join(str(len(_)) for _ in strings)}.csv'
sub.to_csv(sub_name, index=False)

# Final thoughts

What one can also try is to use smarter rebalancing of permutations between strings, instead of just moving duplicates around we can somehow find permutations in one string that 'belong' more to the other string.

That's it, thank you for reading, please upvote if you find it useful.